In [5]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon, LineString, Point
from shapely.ops import unary_union
from shapely import wkt
import folium
from PIL import Image, UnidentifiedImageError
import osmnx as ox
from IPython.display import display
import os
import sys
sys.path.append(os.path.abspath(".."))
from src.gvi_funcs import greenviewindex
from src.gvi_funcs import get_edges
from src.gvi_funcs import extract_edge_data

In [9]:
def load_image(filename):
    file_path = os.path.join(image_dir, filename)
    if os.path.exists(file_path):
        try:
            return Image.open(file_path)
        except UnidentifiedImageError:
            print(f"Warning: Unidentifiable image file {file_path}")
            return None 
    else:
        return None  

In [10]:
df = pd.read_csv('../data/image_metadata_xx.csv')
image_dir = '../data/images_xx'

In [ ]:
df["image"] = df["filename"].apply(load_image)

In [12]:
df

,Filename,Date,Point,OriginalMode,Image
0,image_0.jpg,2022-09-01,POINT (13.4477389 52.526513),RGB,<PIL.JpegImagePlugin.JpegImageFile image mode=...
1,image_1.jpg,2022-09-01,POINT (13.4477389 52.526513),RGB,<PIL.JpegImagePlugin.JpegImageFile image mode=...
2,image_2.jpg,2022-09-01,POINT (13.4477389 52.526513),RGB,<PIL.JpegImagePlugin.JpegImageFile image mode=...
3,image_3.jpg,2022-09-01,POINT (13.4477389 52.526513),RGB,<PIL.JpegImagePlugin.JpegImageFile image mode=...
4,image_4.jpg,2022-08-01,POINT (13.44700535080915 52.52719249588597),RGB,<PIL.JpegImagePlugin.JpegImageFile image mode=...
5,image_5.jpg,2022-08-01,POINT (13.44700535080915 52.52719249588597),RGB,<PIL.JpegImagePlugin.JpegImageFile image mode=...
6,image_6.jpg,2022-08-01,POINT (13.44700535080915 52.52719249588597),RGB,<PIL.JpegImagePlugin.JpegImageFile image mode=...
7,image_7.jpg,2022-08-01,POINT (13.44700535080915 52.52719249588597),RGB,<PIL.JpegImagePlugin.JpegImageFile image mode=...
8,image_8.jpg,2022-08-01,POINT (13.4287152 52.5375413),RGB,<PIL.JpegImagePlugin.JpegImageFile image mode=...
9,image_9.jpg,2022-08-01,POINT (13.4287152 52.5375413),RGB,<PIL.JpegImagePlugin.JpegImageFile image mode=...


In [7]:
df['gvi'] = greenviewindex(df.Image)

In [8]:
df

,Filename,Date,Point,OriginalMode,Image,gvi
0,image_0.jpg,2022-09-01,POINT (13.4477389 52.526513),RGB,<PIL.JpegImagePlugin.JpegImageFile image mode=...,10.266875
1,image_1.jpg,2022-09-01,POINT (13.4477389 52.526513),RGB,<PIL.JpegImagePlugin.JpegImageFile image mode=...,12.220625
2,image_2.jpg,2022-09-01,POINT (13.4477389 52.526513),RGB,<PIL.JpegImagePlugin.JpegImageFile image mode=...,4.751875
3,image_3.jpg,2022-09-01,POINT (13.4477389 52.526513),RGB,<PIL.JpegImagePlugin.JpegImageFile image mode=...,7.033750
4,image_4.jpg,2022-08-01,POINT (13.44700535080915 52.52719249588597),RGB,<PIL.JpegImagePlugin.JpegImageFile image mode=...,9.963125
5,image_5.jpg,2022-08-01,POINT (13.44700535080915 52.52719249588597),RGB,<PIL.JpegImagePlugin.JpegImageFile image mode=...,35.137500
6,image_6.jpg,2022-08-01,POINT (13.44700535080915 52.52719249588597),RGB,<PIL.JpegImagePlugin.JpegImageFile image mode=...,42.085625
7,image_7.jpg,2022-08-01,POINT (13.44700535080915 52.52719249588597),RGB,<PIL.JpegImagePlugin.JpegImageFile image mode=...,23.599375
8,image_8.jpg,2022-08-01,POINT (13.4287152 52.5375413),RGB,<PIL.JpegImagePlugin.JpegImageFile image mode=...,12.915000
9,image_9.jpg,2022-08-01,POINT (13.4287152 52.5375413),RGB,<PIL.JpegImagePlugin.JpegImageFile image mode=...,4.090000


In [ ]:
df['Point'] = df['Point'].apply(wkt.loads)

In [ ]:
gdf = gpd.GeoDataFrame(df, crs='epsg:4326', geometry='Point')

In [ ]:
gdf['lon'] = gdf['Point'].apply(lambda x: x.x)
gdf['lat']  = gdf['Point'].apply(lambda x: x.y)

In [ ]:
gdf['Date'] = pd.to_datetime(gdf['Date'])
gdf

#### add street names & street lentgh

In [ ]:
polygon = [[ 13.335820607449087, 52.479180814784826 ], [ 13.319144034173492, 52.481940744889087 ], [ 13.319144034173492, 52.481940744889087 ], [ 13.303192529301191, 52.491654321569996 ], [ 13.287966092832171, 52.502689873200431 ], [ 13.293041571655179, 52.511516320373843 ], [ 13.288691161235455, 52.519899803498745 ], [ 13.29014129804203, 52.526958338172669 ], [ 13.301017324091331, 52.528281687156365 ], [ 13.324219512996502, 52.530046090443768 ], [ 13.353222249127967, 52.534015738673432 ], [ 13.36337320677398, 52.538426038072501 ], [ 13.380774848452859, 52.545481595702277 ], [ 13.398176490131736, 52.54636346067177 ], [ 13.411227721390897, 52.546804386512193 ], [ 13.42355388424677, 52.541953958639048 ], [ 13.432979773489496, 52.539749041508585 ], [ 13.439505389119075, 52.537985028067389 ], [ 13.44675607315194, 52.534456788548354 ], [ 13.45183155197495, 52.526958338172669 ], [ 13.456907030797952, 52.524311520584895 ], [ 13.462707578024245, 52.523208632849773 ], [ 13.469958262057109, 52.518796804998914 ], [ 13.473583604073546, 52.513943282616047 ], [ 13.473583604073546, 52.509309874748915 ], [ 13.470683330460398, 52.505117322867868 ], [ 13.466695454242322, 52.500703678312291 ], [ 13.464882783234104, 52.497834571715572 ], [ 13.461257441217672, 52.494744555219448 ], [ 13.45799463340288, 52.492316532775945 ], [ 13.457632099201236, 52.489115752903984 ], [ 13.457088297898771, 52.486797801367324 ], [ 13.457088297898771, 52.484038175975996 ], [ 13.456000695293843, 52.4796224152363 ], [ 13.454913092688912, 52.473770849536344 ], [ 13.452375353277407, 52.472114604750061 ], [ 13.444218333740436, 52.470789564040665 ], [ 13.439505389119072, 52.469685332974741 ], [ 13.432073437985386, 52.468470646803397 ], [ 13.426816692061557, 52.46791850564356 ], [ 13.420834877734441, 52.467145496383672 ], [ 13.415396864709793, 52.466262040606331 ], [ 13.408508714878566, 52.465489002255779 ], [ 13.402889434753099, 52.465709871741218 ], [ 13.39781395593009, 52.467366357557452 ], [ 13.394732415216122, 52.468912354744376 ], [ 13.389294402191474, 52.470789564040665 ], [ 13.383131320763537, 52.471728138664126 ], [ 13.378055841940533, 52.472335440989724 ], [ 13.374067965722457, 52.472280232033704 ], [ 13.370533257256433, 52.472280232033704 ], [ 13.368720586248219, 52.472997943059788 ], [ 13.367361082992057, 52.473770849536344 ], [ 13.366545381038357, 52.475427031989611 ], [ 13.365185877782197, 52.477248760694756 ], [ 13.363645107425214, 52.478352801991761 ], [ 13.361379268664942, 52.47956721542225 ], [ 13.35893216280385, 52.480340006516634 ], [ 13.355034920136184, 52.480395205361077 ], [ 13.351500211670158, 52.479953612666179 ], [ 13.348781205157836, 52.478960012896962 ], [ 13.346062198645511, 52.478628807987505 ], [ 13.343886993435651, 52.479015213472806 ], [ 13.341258620473734, 52.479346415473572 ], [ 13.339174048814286, 52.479291215313218 ], [ 13.335820607449087, 52.479180814784826 ]]

In [ ]:
polygon = Polygon(polygon)

In [ ]:
G = ox.graph_from_polygon(polygon=polygon, simplify=True, network_type='drive')

In [4]:
def get_edges(G, lon: pd.Series, lat: pd.Series) -> list:
    try:
        edges = ox.nearest_edges(G, [lon], [lat])
        edges = list(dict.fromkeys(edges))  
        return [k[0:2] for k in edges]
    except Exception as e:
        return []

In [ ]:
gdf['edges'] = gdf.apply(lambda row: get_edges(G, row['lon'], row['lat']), axis=1)

In [ ]:
gdf['edges'] = gdf['edges'].apply(lambda edges: [tuple(edge) for edge in edges])

In [3]:
def extract_edge_data(edge_list: list, G):

    if not edge_list:
        return None, None
    
    u, v = edge_list[0]
    edge_data = G.get_edge_data(u, v)
    
    if edge_data is not None:
        data = edge_data.get(0, {})
        geometry = data.get('geometry', None)
        name = data.get('name', None)
        
        if not isinstance(geometry, LineString):
            geometry = None
        
        return geometry, name
    else:
        return None, None

In [ ]:
gdf[['edge_geometry', 'edge_name']] = gdf['edges'].apply(lambda edges: pd.Series(extract_edge_data(edges, G)))

In [ ]:
gdf.rename(columns={'edge_geometry': 'geometry'}, inplace=True)
gdf.set_geometry('geometry', inplace=True)

In [ ]:
gdf['length'] = gdf['geometry'].apply(lambda x: (x.length*1000))

In [ ]:
# set GVI in relation to total length of street for making data more relvant.
# This assumes that people prefer a least turn route choice while byking.
gdf['ratio'] = (gdf['gvi']/gdf['length'])

In [ ]:
gdf.head(2)

#### take into accout bike lane network

In [ ]:
# https://gdi.berlin.de/geonetwork/srv/ger/catalog.search#/metadata/0c37901e-f16f-3c4d-aa0b-10e1bde35af2
bike_df =gpd.read_file('../data/bike_network.json')

In [ ]:
bike_df.set_index('gisid', inplace=True)

In [ ]:
#set buffer for sjoin --> merge bike lane data with gvi
gdf['buffer'] = gdf['geometry'].buffer(distance=0.0001)

In [ ]:
m = folium.Map(location=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()], zoom_start=10)

for _, row in gdf.iterrows():
    folium.GeoJson(row['buffer'].__geo_interface__).add_to(m)
    
for _, row in bike_df.iterrows():
    folium.GeoJson(row['geometry'].__geo_interface__, style_function=lambda x: {'color': 'green'}).add_to(m)

m

In [ ]:
result = gpd.sjoin(gdf, bike_df, how='left',predicate = "intersects" )

In [ ]:
# for droping duplicates
result.drop(['elem_nr', 'index_right'], inplace=True, axis=1)

In [ ]:
result

In [ ]:
# multiple edge lane starts intersect with a single bikelane 
#result.drop_duplicates(keep='first', inplace=True)

####  Berlin's Least Inviting Streets for Cyclists

In [ ]:
top10 = result

In [ ]:
top10 = top10[top10['existenz'] != 'ja, in Straßennetz']

In [ ]:
top10 = top10.groupby('edge_name').agg({'gvi': 'mean', 'length':'sum', 'ratio':'mean'}).reset_index()

In [ ]:
# get a certain length for a more represantive bike activity
#top10 = top10[top10['length']>10]

In [ ]:
top10.nsmallest(10, 'ratio')